# Text Classifier DSPy Program

Based on [Build generative AI apps using DSPy on Databricks](https://docs.databricks.com/aws/en/generative-ai/dspy)

This demo notebook shows how to use DSPy on Databricks to build and optimize AI agents (_agentic systems_).

DSPy comes with the following features:

1. Automate prompt engineering (replacing manual text-templating prompt engineering with structured Python-based "text transformation graphs").
1. Orchestrate LLM fine-tuning to improve performance.

DSPy consists of the following building blocks for agent development and improving agent quality:

1. **Modules**: In DSPy, these are components that handle specific text transformations, like answering questions or summarizing. They replace traditional hand-written prompts and can learn from examples, making them more adaptable.
1. **Signatures**: A natural language description of a module's input and output behavior. For example, “question -> answer” specifies that the module should take a question as input and return an answer.
1. **Compiler**: This is DSPy's optimization tool. It improves LM pipelines by adjusting modules to meet a performance metric, either by generating better prompts or fine-tuning models.
1. **Program (DSPy)**: A set of modules connected into a pipeline to perform complex tasks. DSPy programs are flexible, allowing you to optimize and adapt them using the compiler.

Let's create a DSPy program for text classification.

## Requirements

In [0]:
%pip install -qU dspy>=3.0.4 mlflow>=3.7.0
dbutils.library.restartPython()

In [0]:
import mlflow
print(f"MLflow version: {mlflow.__version__}")

In [0]:
import dspy
print(f"DSPy version: {dspy.__version__}")

## Choose LM

We'll use one of the Databricks foundational model serving endpoints.

Let's list the available serving endpoints (using MLflow).

In [0]:
from mlflow.deployments import get_deploy_client
from pyspark.sql.types import StructType, StructField, StringType, MapType

deploy_client = get_deploy_client("databricks")
endpoints = deploy_client.list_endpoints()

endpoint_names = sorted([endpoint['name'] for endpoint in endpoints])

# Wrap each name in a tuple to match the schema to build a DataFrame from
endpoint_names_df = spark.createDataFrame(
    [(name,) for name in endpoint_names],
    schema=StructType([StructField("Endpoint Name", StringType())])
)

display(endpoint_names_df)

In [0]:
import dspy

model_serving_endpoint_name = "databricks-gpt-5-2"

# Leverage default authentication inside the Databricks context (notebooks, workflows, etc.)
lm = dspy.LM(
    model=f"databricks/{model_serving_endpoint_name}",
)
dspy.settings.configure(lm=lm)

## Dataset

The following downloads the [Reuters 21578](https://huggingface.co/datasets/yangwang825/reuters-21578) dataset from Huggingface and defines a utility to ensure that your train and test splits have the same labels.

Do I understand what the code does?! Not really 🤷‍♂️

In [0]:
import numpy as np
import pandas as pd

from dspy.datasets.dataset import Dataset
from sklearn.model_selection import StratifiedShuffleSplit


def read_data_and_subset_to_categories() -> tuple[pd.DataFrame]:
    """
    Read the reuters-21578 dataset. Docs can be found in the url below:
    https://huggingface.co/datasets/yangwang825/reuters-21578
    """

    # Read train/test split
    file_path = "hf://datasets/yangwang825/reuters-21578/{}.json"
    train = pd.read_json(file_path.format("train"))
    test = pd.read_json(file_path.format("test"))

    # Clean the labels
    label_map = {
        0: "acq",
        1: "crude",
        2: "earn",
        3: "grain",
        4: "interest",
        5: "money-fx",
        6: "ship",
        7: "trade",
    }

    train["label"] = train["label"].map(label_map)
    test["label"] = test["label"].map(label_map)

    return train, test


class CSVDataset(Dataset):
    def __init__(
        self,
        n_train_per_label: int = 20,
        n_test_per_label: int = 10,
        *args,
        **kwargs
    ) -> None:
        super().__init__(*args, **kwargs)
        self.n_train_per_label = n_train_per_label
        self.n_test_per_label = n_test_per_label

        self._create_train_test_split_and_ensure_labels()

    def _create_train_test_split_and_ensure_labels(self) -> None:
        """Perform a train/test split that ensure labels in `dev` are also in `train`."""
        # Read the data
        train_df, test_df = read_data_and_subset_to_categories()

        # Sample for each label
        train_samples_df = pd.concat([
            group.sample(n=self.n_train_per_label) 
            for _, group in train_df.groupby('label')
        ])
        test_samples_df = pd.concat([
            group.sample(n=self.n_test_per_label) 
            for _, group in test_df.groupby('label')
        ])

        # Set DSPy class variables
        self._train = train_samples_df.to_dict(orient="records")
        self._dev = test_samples_df.to_dict(orient="records")


# Limit to a small dataset to showcase the value of bootstrapping
dataset = CSVDataset(n_train_per_label=3, n_test_per_label=1)

# Create train and test sets containing DSPy
# Note that we must specify the expected input value name
train_dataset = [example.with_inputs("text") for example in dataset.train]
test_dataset = [example.with_inputs("text") for example in dataset.dev]

print(len(train_dataset), len(test_dataset))
print(f"Train labels: {set([example.label for example in dataset.train])}")
print(train_dataset[0])

## DSPy Program

Finally, define your text classification task.

There are a variety of ways you can provide guidelines to DSPy signature behavior. Currently, DSPy allows users to specify:

1. A high-level goal via the class docstring.
1. A set of input fields, with optional metadata.
1. A set of output fields with optional metadata.

DSPy leverages this information to inform optimization.

In the following example, notice that there is not information in the `TextClassificationSignature` class about the dataset of interest. This means you are effectively leveraging a base LLM instructed to classify text with no context. From this blank slate, you can use DSPy to learn to classify from training alone.

Note that in production, you should provide metadata in the signature to reduce training time. This secenario is just for demonstration purposes.

In [0]:
import dspy

class TextClassificationSignature(dspy.Signature):
    text = dspy.InputField()
    label = dspy.OutputField(desc="Label of predicted_class")


# FIXME This superfluous module is likely not needed
class TextClassifier(dspy.Module):
    def __init__(self):
      super().__init__()
      self.generate_classification = dspy.Predict(TextClassificationSignature)

    def forward(self, text: str):
      return self.generate_classification(text=text)


## Test Classification

The DSPy program incorrectly classifies the text because it has not been given the labels.

In [0]:
text_classifier = TextClassifier()

message = "I am interested in space"
print(text_classifier(text=message))

message = "I enjoy ice skating"
print(text_classifier(text=message))

## DSPy Training

For training, you can use `BootstrapFewShotWithRandomSearch`, an optimizer that takes bootstrap samples from your training set and leverages a random search strategy to optimize your predictive accuracy.

Note that the following example uses the simple metric definition of exact match, as defined in `validate_classification`, but `dspy.Metrics` can contain complex and LM-based logic to properly evaluate our accuracy.

In [0]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

def validate_classification(example, prediction, trace=None) -> bool:
    return example.label == prediction.label

optimizer = BootstrapFewShotWithRandomSearch(
    metric=validate_classification,
    num_candidate_programs=5,
    max_bootstrapped_demos=2,
    num_threads=1,
)

compiled_pe = optimizer.compile(TextClassifier(), trainset=train_dataset)


## Understand DSPy Training

Understanding the path that DSPy training returns can be challenging, however there are a few APIs exposed.

The following shows a way to print a formatted and color-coded message history for a number of messages.

In [0]:
lm.inspect_history(n=1)


⚠️ There's [MLflow Tracing](https://mlflow.org/docs/latest/genai/tracing/), too!

## Compare Model Accuracy

Let's review how well your newly-trained model can predict on unseen test data.

In [0]:
def check_accuracy(classifier, test_data: pd.DataFrame = test_dataset) -> float:
    residuals = []
    predictions = []
    for example in test_data:
        prediction = classifier(text=example["text"])
        residuals.append(int(validate_classification(example, prediction)))
        predictions.append(prediction)
    return residuals, predictions

uncompiled_residuals, uncompiled_predictions = check_accuracy(TextClassifier())
print(f"Uncompiled accuracy: {np.mean(uncompiled_residuals)}")
  
compiled_residuals, compiled_predictions = check_accuracy(compiled_pe)
print(f"Compiled accuracy: {np.mean(compiled_residuals)}")


I've got no idea what these accuracy numbers really mean.

Quoting the source notebook (with the results adjusted):

> As shown above, your uncompiled accuracy is 0 - your base LLM was not aware of the classification labels.
>
> However, with training alone, the prompts, demonstrations, and input and output signatures have been updated to prompt your model to reach 100% accuracy (astonishing!)

In [0]:
for uncompiled_residual, uncompiled_prediction in zip(uncompiled_residuals, uncompiled_predictions):
    is_correct = "Correct" if bool(uncompiled_residual) else "Incorrect"
    prediction = uncompiled_prediction.label
    print(f"{is_correct} prediction: {' ' * (12 - len(is_correct))}{prediction}")

In [0]:
for compiled_residual, compiled_prediction in zip(compiled_residuals, compiled_predictions):
    is_correct = "Correct" if bool(compiled_residual) else "Incorrect"
    prediction = compiled_prediction.label
    print(f"{is_correct} prediction: {' ' * (12 - len(is_correct))}{prediction}")